# Regularization - Equal Thickness Lenses

In this example, we design a two lens system where the first lens is plano-convex, and the second lens is biconvex symmetric. Two parameters are used to describe the curvature of each lens.

This problem has many solutions because different shape combinations can achieve the desired focal length. Using regularization, we add the additional constraint that the inner thickness of each lens should be equal. This leads to a unique solution.

In [ ]:
import math
import torch
import torch.nn as nn
import numpy as np

import torch.optim as optim
import torchlensmaker as tlm


class Optics(tlm.Module):
    def __init__(self):
        super().__init__()

        lens_radius = 30
        focal_length = 45
        lens_outer_thickness = 1.0
        material = (1.0, 1.5) # air and glass

        # Shape of the curved surface of the plano convex lens
        self.shape_convex = tlm.Parabola(lens_radius, nn.Parameter(torch.tensor(0.005)))

        # Shape of the two curved surfaces of the biconvex symmetric lens
        self.shape_biconvex = tlm.Parabola(lens_radius, nn.Parameter(torch.tensor(0.005)))
    
        self.plano = tlm.PlanoLens(
            self.shape_convex,
            n = material,
            outer_thickness = lens_outer_thickness,
        )
        
        self.biconvex = tlm.SymmetricLens(
            self.shape_biconvex,
            n = material,
            outer_thickness = lens_outer_thickness,
        )

        self.optics = nn.Sequential(
            tlm.PointSourceAtInfinity(0.9*lens_radius),
            tlm.Gap(10.),
            self.plano,
            tlm.Gap(3.),
            self.biconvex,
            tlm.Gap(focal_length),
            tlm.FocalPointLoss(),
        )

    def forward(self, inputs):
        return self.optics(inputs)


def regu_equalthickness(optics):
    t0 = optics.plano.inner_thickness()
    t1 = optics.biconvex.inner_thickness()
    return 100*torch.pow(t0 - t1, 2)


optics = Optics()

print(list(optics.named_parameters()))

tlm.render_plt(optics, force_uniform_source=False)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=5e-4),
    num_iter = 250,
    regularization = regu_equalthickness,
)

tlm.render_plt(optics, force_uniform_source=False)

print(f"Plano-convex inner thickness {optics.plano.inner_thickness():.4f}")
print(f"Bi-convex inner thickness {optics.biconvex.inner_thickness():.4f}")

In [ ]:
from IPython.display import display

display(tlm.lens_to_part(optics.plano))
display(tlm.lens_to_part(optics.biconvex))